In [1]:
import pandas as pd
import sqlite3
import numpy as np
from sqlalchemy import create_engine
import os
import datetime as dt
from sklearn.model_selection import train_test_split

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve

from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math


from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss


In [2]:
df = pd.read_csv("final_features.csv")
df.head()

,Unnamed: 0,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,...,374_y,375_y,376_y,377_y,378_y,379_y,380_y,381_y,382_y,383_y
0,0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,...,16.165592,33.030668,7.019996,-14.793959,15.437511,8.199658,-25.070834,1.571619,1.603738,0.305645
1,1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,...,-4.901128,-4.565393,41.520751,-0.727564,-16.413776,-7.373778,2.638877,-7.403457,2.703070,0.408040
2,2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,...,8.359966,-2.165985,10.936580,-16.531660,14.681230,15.633759,-1.210901,14.183826,11.703135,10.148075
3,3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,3.311411,3.788879,13.398598,-6.592596,6.437365,5.993293,2.732392,-3.727647,5.614115,6.023693
4,4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,...,-2.403870,11.991204,8.088483,-15.090201,8.375166,1.727225,-6.601129,11.317413,11.544603,2.478689


In [ ]:
if not os.path.isfile("train.db"):
    disk_engine = create_engine("sqlite:///train.db")
    start = dt.datetime.now()
    chunksize = 180000
    j = 0
    index_start = 1
    for df in pd.read_csv('final_features.csv', names=['Unnamed: 0','id','is_duplicate','cwc_min','cwc_max','csc_min','csc_max','ctc_min','ctc_max','last_word_eq','first_word_eq','abs_len_diff','mean_len','token_set_ratio','token_sort_ratio','fuzz_ratio','fuzz_partial_ratio','longest_substr_ratio','freq_qid1','freq_qid2','q1len','q2len','q1_n_words','q2_n_words','word_Common','word_Total','word_share','freq_q1+q2','freq_q1-q2','0_x','1_x','2_x','3_x','4_x','5_x','6_x','7_x','8_x','9_x','10_x','11_x','12_x','13_x','14_x','15_x','16_x','17_x','18_x','19_x','20_x','21_x','22_x','23_x','24_x','25_x','26_x','27_x','28_x','29_x','30_x','31_x','32_x','33_x','34_x','35_x','36_x','37_x','38_x','39_x','40_x','41_x','42_x','43_x','44_x','45_x','46_x','47_x','48_x','49_x','50_x','51_x','52_x','53_x','54_x','55_x','56_x','57_x','58_x','59_x','60_x','61_x','62_x','63_x','64_x','65_x','66_x','67_x','68_x','69_x','70_x','71_x','72_x','73_x','74_x','75_x','76_x','77_x','78_x','79_x','80_x','81_x','82_x','83_x','84_x','85_x','86_x','87_x','88_x','89_x','90_x','91_x','92_x','93_x','94_x','95_x','96_x','97_x','98_x','99_x','100_x','101_x','102_x','103_x','104_x','105_x','106_x','107_x','108_x','109_x','110_x','111_x','112_x','113_x','114_x','115_x','116_x','117_x','118_x','119_x','120_x','121_x','122_x','123_x','124_x','125_x','126_x','127_x','128_x','129_x','130_x','131_x','132_x','133_x','134_x','135_x','136_x','137_x','138_x','139_x','140_x','141_x','142_x','143_x','144_x','145_x','146_x','147_x','148_x','149_x','150_x','151_x','152_x','153_x','154_x','155_x','156_x','157_x','158_x','159_x','160_x','161_x','162_x','163_x','164_x','165_x','166_x','167_x','168_x','169_x','170_x','171_x','172_x','173_x','174_x','175_x','176_x','177_x','178_x','179_x','180_x','181_x','182_x','183_x','184_x','185_x','186_x','187_x','188_x','189_x','190_x','191_x','192_x','193_x','194_x','195_x','196_x','197_x','198_x','199_x','200_x','201_x','202_x','203_x','204_x','205_x','206_x','207_x','208_x','209_x','210_x','211_x','212_x','213_x','214_x','215_x','216_x','217_x','218_x','219_x','220_x','221_x','222_x','223_x','224_x','225_x','226_x','227_x','228_x','229_x','230_x','231_x','232_x','233_x','234_x','235_x','236_x','237_x','238_x','239_x','240_x','241_x','242_x','243_x','244_x','245_x','246_x','247_x','248_x','249_x','250_x','251_x','252_x','253_x','254_x','255_x','256_x','257_x','258_x','259_x','260_x','261_x','262_x','263_x','264_x','265_x','266_x','267_x','268_x','269_x','270_x','271_x','272_x','273_x','274_x','275_x','276_x','277_x','278_x','279_x','280_x','281_x','282_x','283_x','284_x','285_x','286_x','287_x','288_x','289_x','290_x','291_x','292_x','293_x','294_x','295_x','296_x','297_x','298_x','299_x','300_x','301_x','302_x','303_x','304_x','305_x','306_x','307_x','308_x','309_x','310_x','311_x','312_x','313_x','314_x','315_x','316_x','317_x','318_x','319_x','320_x','321_x','322_x','323_x','324_x','325_x','326_x','327_x','328_x','329_x','330_x','331_x','332_x','333_x','334_x','335_x','336_x','337_x','338_x','339_x','340_x','341_x','342_x','343_x','344_x','345_x','346_x','347_x','348_x','349_x','350_x','351_x','352_x','353_x','354_x','355_x','356_x','357_x','358_x','359_x','360_x','361_x','362_x','363_x','364_x','365_x','366_x','367_x','368_x','369_x','370_x','371_x','372_x','373_x','374_x','375_x','376_x','377_x','378_x','379_x','380_x','381_x','382_x','383_x','0_y','1_y','2_y','3_y','4_y','5_y','6_y','7_y','8_y','9_y','10_y','11_y','12_y','13_y','14_y','15_y','16_y','17_y','18_y','19_y','20_y','21_y','22_y','23_y','24_y','25_y','26_y','27_y','28_y','29_y','30_y','31_y','32_y','33_y','34_y','35_y','36_y','37_y','38_y','39_y','40_y','41_y','42_y','43_y','44_y','45_y','46_y','47_y','48_y','49_y','50_y','51_y','52_y','53_y','54_y','55_y','56_y','57_y','58_y','59_y','60_y','61_y','62_y','63_y','64_y','65_y','66_y','67_y','68_y','69_y','70_y','71_y','72_y','73_y','74_y','75_y','76_y','77_y','78_y','79_y','80_y','81_y','82_y','83_y','84_y','85_y','86_y','87_y','88_y','89_y','90_y','91_y','92_y','93_y','94_y','95_y','96_y','97_y','98_y','99_y','100_y','101_y','102_y','103_y','104_y','105_y','106_y','107_y','108_y','109_y','110_y','111_y','112_y','113_y','114_y','115_y','116_y','117_y','118_y','119_y','120_y','121_y','122_y','123_y','124_y','125_y','126_y','127_y','128_y','129_y','130_y','131_y','132_y','133_y','134_y','135_y','136_y','137_y','138_y','139_y','140_y','141_y','142_y','143_y','144_y','145_y','146_y','147_y','148_y','149_y','150_y','151_y','152_y','153_y','154_y','155_y','156_y','157_y','158_y','159_y','160_y','161_y','162_y','163_y','164_y','165_y','166_y','167_y','168_y','169_y','170_y','171_y','172_y','173_y','174_y','175_y','176_y','177_y','178_y','179_y','180_y','181_y','182_y','183_y','184_y','185_y','186_y','187_y','188_y','189_y','190_y','191_y','192_y','193_y','194_y','195_y','196_y','197_y','198_y','199_y','200_y','201_y','202_y','203_y','204_y','205_y','206_y','207_y','208_y','209_y','210_y','211_y','212_y','213_y','214_y','215_y','216_y','217_y','218_y','219_y','220_y','221_y','222_y','223_y','224_y','225_y','226_y','227_y','228_y','229_y','230_y','231_y','232_y','233_y','234_y','235_y','236_y','237_y','238_y','239_y','240_y','241_y','242_y','243_y','244_y','245_y','246_y','247_y','248_y','249_y','250_y','251_y','252_y','253_y','254_y','255_y','256_y','257_y','258_y','259_y','260_y','261_y','262_y','263_y','264_y','265_y','266_y','267_y','268_y','269_y','270_y','271_y','272_y','273_y','274_y','275_y','276_y','277_y','278_y','279_y','280_y','281_y','282_y','283_y','284_y','285_y','286_y','287_y','288_y','289_y','290_y','291_y','292_y','293_y','294_y','295_y','296_y','297_y','298_y','299_y','300_y','301_y','302_y','303_y','304_y','305_y','306_y','307_y','308_y','309_y','310_y','311_y','312_y','313_y','314_y','315_y','316_y','317_y','318_y','319_y','320_y','321_y','322_y','323_y','324_y','325_y','326_y','327_y','328_y','329_y','330_y','331_y','332_y','333_y','334_y','335_y','336_y','337_y','338_y','339_y','340_y','341_y','342_y','343_y','344_y','345_y','346_y','347_y','348_y','349_y','350_y','351_y','352_y','353_y','354_y','355_y','356_y','357_y','358_y','359_y','360_y','361_y','362_y','363_y','364_y','365_y','366_y','367_y','368_y','369_y','370_y','371_y','372_y','373_y','374_y','375_y','376_y','377_y','378_y','379_y','380_y','381_y','382_y','383_y'], chunksize=chunksize, iterator=True, encoding='utf-8', ):
        df.index += index_start
        j+=1
        df.to_sql("data" , disk_engine , if_exists = "append")
        index_start = df.index[-1]+1
        
    

C:\Users\Acer\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None


def checkTableExists(dbcon):
    cursr = dbcon.cursor()
    str = "select name from sqlite_master where type='table'"
    table_names = cursr.execute(str)
    print("Tables in the databse:")
    tables =table_names.fetchall() 
    print(tables[0][0])
    return(len(tables))


        

In [3]:
read_db = "train.db"
conn_r = create_connection(read_db)
checkTableExists(conn_r)
conn_r.close()


Tables in the databse:
data


In [4]:
if os.path.isfile(read_db):
    conn_r = create_connection(read_db)
    if conn_r is not None:
        data = pd.read_sql_query("SELECT * FROM data ORDER BY RANDOM() LIMIT 100000;",conn_r)
        conn_r.commit()
        conn_r.close()

In [5]:
data.head()

,index,Unnamed: 0,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,...,374_y,375_y,376_y,377_y,378_y,379_y,380_y,381_y,382_y,383_y
0,116211,116209.0,116209,1,0.66664444518516,0.399992000159997,0.0,0.0,0.333327777870369,0.222219753113854,...,15.3497749865055,3.40774642210454,8.25629800371826,0.820842266082764,2.1575875878334,-4.49965637922287,1.45333111099899,-2.47508931159973,-0.787002667784691,2.58603611588478
1,58520,58518.0,58518,1,0.66664444518516,0.499987500312492,0.999975000624984,0.571420408279882,0.749990625117186,0.599994000059999,...,1.17822023108602,3.68825466185808,7.88275320082903,-6.05888026952743,-7.68546399474144,-2.28946168348193,-16.1642403379083,12.5152367353439,6.01604044437408,4.06151978671551
2,918603,378601.0,378601,1,0.999950002499875,0.999950002499875,0.0,0.0,0.399992000159997,0.249996875039062,...,9.80755677819252,0.470452412962914,-3.8624832034111,1.35997123643756,3.19469994306564,1.96701614558697,-0.193701267242432,-2.9942782074213,0.731079936027527,3.63069136440754
3,509973,329971.0,329971,0,0.66664444518516,0.66664444518516,0.0,0.0,0.399992000159997,0.285710204139941,...,7.6310792863369,5.0989630818367,1.31140718668757,-1.17653158307075,1.21864739060402,-3.56833586841822,-14.4745216965675,-5.93922147154808,-5.57844930887222,4.78095416724682
4,148507,148505.0,148505,1,0.499987500312492,0.499987500312492,0.33332222259258,0.166663888935184,0.428565306209911,0.272724793410969,...,-2.31708432920277,9.92692351341248,7.01830238103867,-8.36391559243202,11.7469453513622,4.19425471127033,-1.6350871771574,10.642592266202,12.5418259650469,-4.37330663204193


In [6]:

 # removing the first row 0:unnamed
data.drop(data.index[0],inplace = True)



In [7]:
y_true = data["is_duplicate"]

In [8]:
data.drop(['Unnamed: 0', 'id','index','is_duplicate'], axis=1, inplace=True)
data.head()

,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,...,374_y,375_y,376_y,377_y,378_y,379_y,380_y,381_y,382_y,383_y
1,0.66664444518516,0.499987500312492,0.999975000624984,0.571420408279882,0.749990625117186,0.599994000059999,1.0,1.0,2.0,9.0,...,1.17822023108602,3.68825466185808,7.88275320082903,-6.05888026952743,-7.68546399474144,-2.28946168348193,-16.1642403379083,12.5152367353439,6.01604044437408,4.06151978671551
2,0.999950002499875,0.999950002499875,0.0,0.0,0.399992000159997,0.249996875039062,1.0,0.0,3.0,6.5,...,9.80755677819252,0.470452412962914,-3.8624832034111,1.35997123643756,3.19469994306564,1.96701614558697,-0.193701267242432,-2.9942782074213,0.731079936027527,3.63069136440754
3,0.66664444518516,0.66664444518516,0.0,0.0,0.399992000159997,0.285710204139941,0.0,0.0,2.0,6.0,...,7.6310792863369,5.0989630818367,1.31140718668757,-1.17653158307075,1.21864739060402,-3.56833586841822,-14.4745216965675,-5.93922147154808,-5.57844930887222,4.78095416724682
4,0.499987500312492,0.499987500312492,0.33332222259258,0.166663888935184,0.428565306209911,0.272724793410969,0.0,0.0,4.0,9.0,...,-2.31708432920277,9.92692351341248,7.01830238103867,-8.36391559243202,11.7469453513622,4.19425471127033,-1.6350871771574,10.642592266202,12.5418259650469,-4.37330663204193
5,0.0,0.0,0.33332222259258,0.14285510206997,0.166663888935184,0.0833326388946759,0.0,0.0,6.0,9.0,...,2.946817740798,13.0145040862262,-1.91774615645409,2.65882804989815,10.3273448050022,0.0720448940992355,-14.1857273876667,7.6274768859148,4.28325040638447,1.3891475982964


In [9]:
data.head()



,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,...,374_y,375_y,376_y,377_y,378_y,379_y,380_y,381_y,382_y,383_y
1,0.66664444518516,0.499987500312492,0.999975000624984,0.571420408279882,0.749990625117186,0.599994000059999,1.0,1.0,2.0,9.0,...,1.17822023108602,3.68825466185808,7.88275320082903,-6.05888026952743,-7.68546399474144,-2.28946168348193,-16.1642403379083,12.5152367353439,6.01604044437408,4.06151978671551
2,0.999950002499875,0.999950002499875,0.0,0.0,0.399992000159997,0.249996875039062,1.0,0.0,3.0,6.5,...,9.80755677819252,0.470452412962914,-3.8624832034111,1.35997123643756,3.19469994306564,1.96701614558697,-0.193701267242432,-2.9942782074213,0.731079936027527,3.63069136440754
3,0.66664444518516,0.66664444518516,0.0,0.0,0.399992000159997,0.285710204139941,0.0,0.0,2.0,6.0,...,7.6310792863369,5.0989630818367,1.31140718668757,-1.17653158307075,1.21864739060402,-3.56833586841822,-14.4745216965675,-5.93922147154808,-5.57844930887222,4.78095416724682
4,0.499987500312492,0.499987500312492,0.33332222259258,0.166663888935184,0.428565306209911,0.272724793410969,0.0,0.0,4.0,9.0,...,-2.31708432920277,9.92692351341248,7.01830238103867,-8.36391559243202,11.7469453513622,4.19425471127033,-1.6350871771574,10.642592266202,12.5418259650469,-4.37330663204193
5,0.0,0.0,0.33332222259258,0.14285510206997,0.166663888935184,0.0833326388946759,0.0,0.0,6.0,9.0,...,2.946817740798,13.0145040862262,-1.91774615645409,2.65882804989815,10.3273448050022,0.0720448940992355,-14.1857273876667,7.6274768859148,4.28325040638447,1.3891475982964


In [10]:
## String to numeric
cols = list(data.columns)
for i in cols:
    data[i] = data[i].apply(pd.to_numeric)
    print(i)

cwc_min
cwc_max
csc_min
csc_max
ctc_min
ctc_max
last_word_eq
first_word_eq
abs_len_diff
mean_len
token_set_ratio
token_sort_ratio
fuzz_ratio
fuzz_partial_ratio
longest_substr_ratio
freq_qid1
freq_qid2
q1len
q2len
q1_n_words
q2_n_words
word_Common
word_Total
word_share
freq_q1+q2
freq_q1-q2
0_x
1_x
2_x
3_x
4_x
5_x
6_x
7_x
8_x
9_x
10_x
11_x
12_x
13_x
14_x
15_x
16_x
17_x
18_x
19_x
20_x
21_x
22_x
23_x
24_x
25_x
26_x
27_x
28_x
29_x
30_x
31_x
32_x
33_x
34_x
35_x
36_x
37_x
38_x
39_x
40_x
41_x
42_x
43_x
44_x
45_x
46_x
47_x
48_x
49_x
50_x
51_x
52_x
53_x
54_x
55_x
56_x
57_x
58_x
59_x
60_x
61_x
62_x
63_x
64_x
65_x
66_x
67_x
68_x
69_x
70_x
71_x
72_x
73_x
74_x
75_x
76_x
77_x
78_x
79_x
80_x
81_x
82_x
83_x
84_x
85_x
86_x
87_x
88_x
89_x
90_x
91_x
92_x
93_x
94_x
95_x
96_x
97_x
98_x
99_x
100_x
101_x
102_x
103_x
104_x
105_x
106_x
107_x
108_x
109_x
110_x
111_x
112_x
113_x
114_x
115_x
116_x
117_x
118_x
119_x
120_x
121_x
122_x
123_x
124_x
125_x
126_x
127_x
128_x
129_x
130_x
131_x
132_x
133_x
134_x
135_x
136

In [11]:
y_true = list(map(int ,y_true.values))

##  Random train test split


In [13]:
X_train , X_test , y_train , y_test = train_test_split(data , y_true , stratify = y_true , test_size = 0.3)

In [15]:
print("number of data points in train data" , X_train.shape)
print("number of data points in test data", X_test.shape)

number of data points in train data (69999, 794)
number of data points in test data (30000, 794)


## Logistic regression with hyperparameter tuning

In [25]:
alpha = [10**x for x in range(-5,2)] #hyperparameter of sgd classifier

log_loss_array = []

for i in alpha:
    clf = SGDClassifier(alpha = i , penalty = "l2" , loss= "log" , random_state = 42)
    clf.fit(X_train,y_train)
    sig_clf = CalibratedClassifierCV(clf , method = "sigmoid")
    sig_clf.fit(X_train,y_train)
    predict_y = sig_clf.predict_proba(X_test)
    log_loss_array.append(log_loss(y_test , predict_y , labels = clf.classes_ , eps = 1e-15))
    
best_alpha = np.argmin(log_loss_array)
clf = SGDClassifier(alpha = alpha[best_alpha] , penalty = "l2" , loss = "log" , random_state = 42 )
clf.fit(X_train , y_train)
sig_clf = CalibratedClassifierCV(clf , method = "sigmoid")
sig_clf.fit(X_train,y_train)

predict_y = sig_clf.predict_proba(X_train)
print("for values of best alpha = ", alpha[best_alpha] , "The train log loss is :" , log_loss(y_train, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_test)
print('For values of best alpha = ', alpha[best_alpha], "The test log loss is:",log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))
predicted_y =np.argmax(predict_y,axis=1)
print("Total number of data points :", len(predicted_y))

    


for values of best alpha =  0.1 The train log loss is : 0.4372734982453181
For values of best alpha =  0.1 The test log loss is: 0.4493973816707165
Total number of data points : 30000


## Linear SVM with hyperparameter tuning

In [ ]:
alpha = [10**x for x in range[-5,2]]
log_error_array = []

for i in alpha:
    clf = SGDClassifier(alpha = i , loss = "hinge" , penalty = "l1" , random_state=42 )
    clf.fit(X_train,y_train)
    sig_clf = CalibratedClassifierCV(clf , method = "sigmoid")
    sig_clf.fit(X_train,y_train)
    predict_y = sig_clf.predict_proba(X_test)
    log_error_array.append(log_loss(y_test , predict_y , labels = clf.classes_ , eps = 1e-15))

best_alpha = np.argmin(log_error_array)
clf = SGDClassifier(alpha=alpha[best_alpha], penalty='l1', loss='hinge', random_state=42)
clf.fit(X_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(X_train, y_train)

predict_y = sig_clf.predict_proba(X_train)
print('For values of best alpha = ', alpha[best_alpha], "The train log loss is:",log_loss(y_train, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_test)
print('For values of best alpha = ', alpha[best_alpha], "The test log loss is:",log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))
predicted_y =np.argmax(predict_y,axis=1)
print("Total number of data points :", len(predicted_y))
    

## XG Boost

In [2]:
import xgboost as xgb

In [ ]:

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_test, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=10)

xgdmat = xgb.DMatrix(X_train,y_train)
predict_y = bst.predict(d_test)
print("The test log loss is:",log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))